In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------
# 1. Transformations
# --------------------------
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --------------------------
# 2. Load Oxford-IIIT Pet
# --------------------------
train_dataset = datasets.OxfordIIITPet(
    root="data",
    split="trainval",
    transform=transform_train,
    target_types="category",
    download=True
)

test_dataset = datasets.OxfordIIITPet(
    root="data",
    split="test",
    transform=transform_test,
    target_types="category",
    download=True
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

num_classes = len(train_dataset.classes)

# --------------------------
# 3. Load pretrained model
# --------------------------
# model = models.resnet18(weights="IMAGENET1K_V1")   # Προεκπαιδευμένα βάρη
model = models.alexnet(weights="IMAGENET1K_V1")  # Εναλλακτικά

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final classifier
# model.fc = nn.Linear(model.fc.in_features, num_classes)
model.classifier[6] = nn.Linear(4096, num_classes)

model = model.to(device)

# --------------------------
# 4. Loss & Optimizer
# --------------------------
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
# optimizer = optim.Adam(model.classifier[6].parameters(), lr=1e-3)
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3
)

# --------------------------
# 5. Training loop
# --------------------------
def train(model, loader):
    model.train()
    total_loss, correct = 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()

    acc = 100 * correct / len(loader.dataset)
    return total_loss / len(loader), acc


def evaluate(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(dim=1) == labels).sum().item()

    acc = 100 * correct / len(loader.dataset)
    return acc


# --------------------------
# 6. Run training
# --------------------------


In [12]:
for epochs in [5, 10, 15]:
    # Επαναφορά μοντέλου (ή φορτώνεις το pretrained)
    model = models.alexnet(weights="IMAGENET1K_V1")
    model.classifier[6] = nn.Linear(4096, num_classes)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.classifier[6].parameters(), lr=1e-3)

    for epoch in range(epochs):
        train_loss, train_acc = train(model, train_loader)

    test_acc = evaluate(model, test_loader)
    print(f"Epochs={epochs} → Test Accuracy: {test_acc:.2f}%")



Epochs=5 → Test Accuracy: 72.12%
Epochs=10 → Test Accuracy: 72.85%
Epochs=15 → Test Accuracy: 72.28%


In [13]:
for lr in [1e-2, 1e-3, 1e-4]:
    model = models.alexnet(weights="IMAGENET1K_V1")
    model.classifier[6] = nn.Linear(4096, num_classes)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.classifier[6].parameters(), lr=lr)

    for epoch in range(5):
        train_loss, train_acc = train(model, train_loader)

    test_acc = evaluate(model, test_loader)
    print(f"LR={lr} → Test Accuracy: {test_acc:.2f}%")


LR=0.01 → Test Accuracy: 69.31%
LR=0.001 → Test Accuracy: 73.04%
LR=0.0001 → Test Accuracy: 74.16%
